#### Contextual Compression
1. When retrieval with vector storage, there may be some cases where the accurate response to your query may be burried deep inside the irrelevant stuff in your vector database
   

2. i.e., you are not able to extract ot retieve the specific relevant data



3. Also passing the full document through your application can lead to more expensive LLM call and poorer responses

4. Idea : instead of immediately returning retrieved document as-is, you can compress them using the context of the given query -> so that relavant information is returned.

5. Compression here = compressing the contents of individual document + filtering out document wholesale.

6. Contextual Compression requires : a base retrieval + a document compressor

7. Contextual Compression Retirever : 1. passes queries to base retiever. 2. takes the initial document and passes them through Document Compressor 3. Document compressor takes a list of document and shortens it by reducing the contents of document or dropping documents together




In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
SECRET_KEY = os.environ.get('OPEN_AI_KEY') 

'yourkeyhere'

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
# %pip install chromadb
from langchain_community.vectorstores import Chroma
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

chat = ChatOpenAI(openai_api_key = SECRET_KEY)

# Load documents
loader = TextLoader('./data/sample.txt')
mydata = loader.load()

# Split document
text_splitter = CharacterTextSplitter(chunk_size = 100, chunk_overlap = 0)
my_document = text_splitter.split_documents(mydata)

# Embbed Model Object
embedding_function = OpenAIEmbeddings(openai_api_key = SECRET_KEY)

# Store - will store your documents vectors
db = Chroma.from_documents(my_document, embedding_function, presist_directory = './chro_db')
db.persist()

# Read From ChromaDB
db_connection = Chroma(embedding_function = embedding_function, presist_directory = './chro_db')


query = "How did India grew ? "

# Compression
compressor = LLMChainExtractor.from_llm(chat)

# Contextual Compression
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=db_connection.as_retriever()
)

# Vector Store Retriever
retriever = db_connection.as_retriever(search_kwargs={'k':1})
similar_docs = retriever.get_relevant_documents(query)
print("Without Compression : ",similar_docs)



# With compressor

similar_docs = compressor_retriever.get_relevant_documents(query)
print(similar_docs.page_content)